In [1]:
#문서 path 추출
import pandas as pd
import os
import re
import json
import numpy as np

filenames = []
directory = '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)'
for filename in os.listdir(directory):
    #filename이 .json으로 끝나면
    if filename.endswith(".json"):
        #f 생성
        f = directory + '/' + filename
        #비어있는 filenames에 append
        filenames.append(f)

print(filenames[:5])
print("파일 개수:", len(filenames))

['/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZHF20002969.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZHF20000954.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZJD21001992.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZJD20000781.json', '/Users/minjikim/Desktop/2021년 하반기 민지 휴학/하울스/제주도 방언 /training/[라벨]제주도_학습용데이터_1 (1)/DZHF20002590.json']
파일 개수: 5042


In [2]:
#방언, 표준어 추출
standard= []
dialect = []
for file in filenames:
    with open(file, "r", encoding = 'utf8') as f:
        contents = f.read() #string type 파일 내용 전체를 문자열로
        json_data = json.loads(contents)
        doc = json_data["utterance"]
        for d in doc:
            standard.append(d['standard_form'])
            dialect.append(d['dialect_form'])

In [6]:
print(standard[:5])
print(dialect[:5])
print(len(standard))
print(len(dialect))

['왜냐하면 이만한 그', '-이만- 이만한 크기.', '응.', '제사 때 쓰는 거.', '아니 게메 저 차판 자체가 쓸 일이 많지가 않을 거 같아 #이 거기서 #이 왜냐하면']
['왜냐하면 이만한 그', '-이만- 이만한 크기.', '응.', '제사 때 쓰는 거.', '아니 게메 저 차판 자체가 쓸 일이 하지가 않을 거 닮아 이 거기서 이 왜냐하면']
2774257
2774257


In [7]:
#dataframe
train = pd.DataFrame({'standard': standard, 'dialect': dialect})

In [8]:
train[:5]

,standard,dialect
0,왜냐하면 이만한 그,왜냐하면 이만한 그
1,-이만- 이만한 크기.,-이만- 이만한 크기.
2,응.,응.
3,제사 때 쓰는 거.,제사 때 쓰는 거.
4,아니 게메 저 차판 자체가 쓸 일이 많지가 않을 거 같아 #이 거기서 #이 왜냐하면,아니 게메 저 차판 자체가 쓸 일이 하지가 않을 거 닮아 이 거기서 이 왜냐하면


In [ ]:
'''
전처리하면서 살펴봐야 하는 목록
1) {laughing}, {cleaning} 등의 비언어적 표현
2) (이)/(#이) 와 같은 표현은 아예 삭제해도 문제 없는 듯함 
3) 2774255에서 보는 바와 같이 (방언 전사 형태)/(표준어 대응쌍 형태)로 구성 되어 있음 
4) 정보 비식별화를 위해 &name&, &address& 등으로 되어 있음
5) 아래아는 아*  형태로 되어 있음
6) (()), ((x)), ((xxxx)) 등의 형태 
'''

In [10]:
# Dialect_form
# 1번 2번 삭제
pre_s = []
for sentence in standard:
    sentence_new = re.sub(r'\{.+\}', '', sentence) #{laughing}, {clearing} 삭제 
    sentence_new = re.sub('\(이\)/\(#이\)', '', sentence_new) # (이)/(#이) 삭제
    pre_s.append(sentence_new)

In [11]:
# standard는 두 번째 그룹 선택 
pre_standard = []
for snts in pre_s:  #(방언)/(표준어)형태의 문장이 있을 경우 (표준어)를 선택하도록 함
    if re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts):
        m = re.search('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', snts)
        #snts에서 (방언)/(표준어)를 찾고 이를 group(1), group(2)로 보는 것 
        replace_str = re.sub('[()]','',m.group(2)) #(표준어)를 표준어로 바꾸는 작업
        sentence_new = re.sub('(\([\s]?[가-힣]+[\.\?\s\*]?\))/(\([\s#]?[가-힣]+[\.\?\s\*]?\))', replace_str, snts)
        
        pre_standard.append(sentence_new)
    
    else: #(방언)/(표준어) 형태의 expression이 없을 때는 원래의 문장을 바로 pre_standard에 append
        pre_standard.append(snts) 